<a href="https://colab.research.google.com/github/andreea-cochintele/IA/blob/master/DeepLearning%20Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1: Bucharest Housing Dataset





## To Do

To complete this assignment, you must:
1. Get the data in a PyTorch-friendly format;
2. Predict the `Nr Camere` of each dwelling, treating it as a **classification** problem. Choose an appropriate loss function;
3. Predict the `Nr Camere` of each dwelling, treating it as a **regression** problem. Choose an appropriate loss function;
4. Compare the results of the two approaches, displaying the Confusion Matrix for the two, as well as any comparing any other metrics you think are interesting (e.g. MSE). Comment on the results;

-- Tratarea problemei propuse ca o problrma de clasificare este mult mai potrivita

5. Choose to predict a feature more suitable to be treated as a **regression** problem, then successfully solve it.

6. What values should the loss have when the predictions are random (when your network is not trained at all)?
-- Daca predictiile ar fi random loss-ul ar fi foarte mare deoarece acuratetea ar fi aproximativ 50%.

7. Don't forget to split the dataset in training and validation.




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive




# Citire date

In [ ]:
!pip install torchviz

  Created wheel for torchviz: filename=torchviz-0.0.2-cp37-none-any.whl size=4152 sha256=c3481ccfb786972e8e002610ef6602450831c31866e6ae633e584a19b4f50249
  Stored in directory: /root/.cache/pip/wheels/01/26/58/026ffd533dbe8b3972eb423da9c7949beca68d1c98ed9e8624
Successfully built torchviz


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
from typing import Iterator
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing


In [ ]:
data = pd.read_csv("/content/gdrive/My Drive/Colab/Bucharest_HousePriceDataset.csv", delimiter=',')
print(data)


      Nr Camere  Suprafata  Etaj  Total Etaje  Sector  Scor    Pret
0             4     108.00     2            3       4     5   83000
1             1      41.00     1            8       1     1   39900
2             3      63.52     1            3       2     3   84900
3             1      33.00     3           10       5     1   45500
4             2      62.00     5            9       5     5   54900
...         ...        ...   ...          ...     ...   ...     ...
3524          3     102.00     2            5       1     1  189000
3525          4      96.00     1            3       3     2  200000
3526          1      35.00     1            3       4     5   47000
3527          2      60.00     6            7       1     2   89500
3528          3      80.00     3            5       1     2  139000

[3529 rows x 7 columns]



# Regresie

In [ ]:
class MSE():
    """The Mean Squared Error loss"""

    def __call__(self, y: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        y = torch.Tensor(y)
        target = torch.Tensor(target)
        mse = torch.sum((y - target) ** 2) / len(y)
        return mse


In [ ]:
class MAE():
    """The Mean Absolute Error loss"""

    def __call__(self, y: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        y = torch.Tensor(y)
        target = torch.Tensor(target)
        mae = torch.sum(abs(y - target))/ len(y)
        return mae


In [ ]:
class GDLinearRegression(nn.Module):
    """A simple Linear Regression model"""

    def __init__(self):
        super().__init__()
        # We're initializing our model with random weights
        self.w = nn.Parameter(torch.randn(6, requires_grad=True))
        self.b = nn.Parameter(torch.randn(1, requires_grad=True))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = torch.Tensor(x)
        y = x.matmul(self.w) + self.b
        return y

    def zero_grad(self):
        self.w.grad.zero_()
        self.b.grad.zero_()

In [ ]:
class GD:
  """
  Gradient Descent optimizer
  """
  def __init__(self, params: Iterator[nn.Parameter], lr: int):
    self.w, self.b = list(params)
    self.lr = lr


  def step(self):
    with torch.no_grad():
      self.w -= self.w.grad * self.lr
      self.b -= self.b.grad * self.lr


In [ ]:
def train(model: GDLinearRegression, data: torch.Tensor,
          target: torch.Tensor, optim: GD, criterion: MSE):
    """Linear Regression train routine"""
    # forward pass: compute predictions 
    predictions = model(data)

    # forward pass: compute loss 
    loss = criterion(target, predictions)

    # backpropagation: compute gradients of loss wrt weights
    loss.backward()
    with torch.no_grad():
        # GD step: update weights using the gradients 
        optim.step()

        # reset the gradients 
        model.zero_grad()

    return model

In [ ]:
# impartim datele si etichetele
X = data.drop(columns='Nr Camere').values
Y = data[['Nr Camere']].values.ravel()
train_data, valid_data, train_label, valid_label = train_test_split(X,Y, train_size = 0.7)


In [ ]:
# Normalizam datele 
std_scale = preprocessing.StandardScaler().fit(train_data)
train_data = std_scale.transform(train_data)
train_data = torch.tensor(train_data).float()

valid_data = std_scale.transform(valid_data)
valid_data = torch.tensor(valid_data).float()

train_label = torch.tensor(train_label).float()
valid_label = torch.tensor(valid_label).float()

In [ ]:
#Alegem un learning rate si numarul de pasi de antrenare
lr = 0.1
total_steps = 500
model = GDLinearRegression()
optimizer = GD(model.parameters(), lr=lr)
criterion = MSE()

for i in range(total_steps):
    train(model, train_data, train_label, optimizer, criterion)



In [ ]:
# Calculam acuratetea pe datele de train(70% din date)
with torch.no_grad():
    pred_label = model(train_data)

predicted = torch.round(pred_label).numpy()
train_label = train_label.numpy()
accuracy = np.sum((predicted == train_label))
accuracy = accuracy / predicted.shape[0]
print("Acuratete pe date de train")
print(accuracy)

Acuratete pe date de train
0.6919028340080972


In [ ]:
# Calculam acuratetea pe datele de validare(30% din date)
with torch.no_grad():
    pred_label = model(valid_data)

predicted = torch.round(pred_label).numpy()
valid_label = valid_label.numpy()
accuracy = np.sum((predicted == valid_label))
accuracy = accuracy / predicted.shape[0]
print("Acuratete pe date de validare")
print(accuracy)

Acuratete pe date de validare
0.6902738432483475


In [ ]:
# Afisam matricea de confuzie 
regression_matrix1 = confusion_matrix(predicted,valid_label)
print(regression_matrix1)

[[ 12   0   0   0   0   0   0   0   0]
 [105 441  97   2   0   0   0   0   0]
 [  0  30 246  40   4   0   0   0   0]
 [  0   0  29  28   7   0   0   0   0]
 [  0   0   0   5   2   0   0   0   0]
 [  0   0   0   5   2   1   0   1   0]
 [  0   0   0   0   0   0   1   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0]]


# Clasificare

In [ ]:
# impartim datele si etichetele
X = data.drop(columns='Nr Camere').values
Y = data[['Nr Camere']].values
train_data2, valid_data2, train_label2, valid_label2 = train_test_split(X,Y, train_size = 0.7)

In [ ]:
# Normalizam datele 

std_scale = preprocessing.StandardScaler().fit(train_data2)
train_data2 = std_scale.transform(train_data2)
train_data2 = torch.tensor(train_data2).float()

valid_data2 = std_scale.transform(valid_data2)
valid_data2 = torch.tensor(valid_data2).float()

train_label2= torch.tensor(train_label2).float()
valid_label2 = torch.tensor(valid_label2).float()


In [ ]:
class Layers(nn.Module):
    """Multi-Layer Neural Network using Pytorch nn.Linear"""

    def __init__(self,
                 input_size: int,
                 hidden_size: int,
                 output_size: int,
                 activation_fn=lambda x: torch.softmax(x, dim=-1)):
        super().__init__()

        # we instantiate each layer
        self._layer1 = nn.Linear(input_size, hidden_size)
        self._layer2 = nn.Linear(hidden_size, output_size)

        # store the activation function
        self._activation_fn = activation_fn

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # step 1: apply linear layer to x
        x = self._layer1(x)
        x = torch.relu(x)
        x = self._layer2(x)

        # step 2: apply softmax to output
       # y = self._activation_fn(x)  //nu aplicam softmax deoarece se va aplica in CrossEntropyLoss

        return x

In [ ]:
#Antrenam modelul
model = Layers(6, 3000, 10)
EPOCHS = 1000
optim = torch.optim.SGD(model.parameters(), lr=2.5)
for i in range(EPOCHS):
    model.train()
    output = model(train_data2)
    criterion = nn.CrossEntropyLoss()
    target = torch.tensor(train_label2).long().squeeze(1)
    loss = criterion(output, target)
    print(loss)
    loss.backward()
    optim.step()
    model.zero_grad()

tensor(2.4511, grad_fn=<NllLossBackward>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(57.0685, grad_fn=<NllLossBackward>)
tensor(66.2485, grad_fn=<NllLossBackward>)
tensor(24.3493, grad_fn=<NllLossBackward>)
tensor(7.5291, grad_fn=<NllLossBackward>)
tensor(9.0626, grad_fn=<NllLossBackward>)
tensor(1.4545, grad_fn=<NllLossBackward>)
tensor(1.2211, grad_fn=<NllLossBackward>)
tensor(1.1073, grad_fn=<NllLossBackward>)
tensor(0.9784, grad_fn=<NllLossBackward>)
tensor(0.9689, grad_fn=<NllLossBackward>)
tensor(0.9493, grad_fn=<NllLossBackward>)
tensor(0.9718, grad_fn=<NllLossBackward>)
tensor(0.9545, grad_fn=<NllLossBackward>)
tensor(0.9593, grad_fn=<NllLossBackward>)
tensor(0.9109, grad_fn=<NllLossBackward>)
tensor(0.8865, grad_fn=<NllLossBackward>)
tensor(0.8353, grad_fn=<NllLossBackward>)
tensor(0.8069, grad_fn=<NllLossBackward>)
tensor(0.7732, grad_fn=<NllLossBackward>)
tensor(0.7521, grad_fn=<NllLossBackward>)
tensor(0.7324, grad_fn=<NllLossBackward>)
tensor(0.7186, grad_fn=<NllLossBackward>)
tensor(0.7077, grad_fn=<NllLossBackward>)
tensor(0.7002, grad_fn=<NllLoss

In [ ]:
# Afisam acuratetea
predicted2 = torch.tensor(torch.argmax(model(train_data2), dim=-1))
accuracy2 = torch.sum(predicted2 == train_label2.ravel())
accuracy2 = accuracy2 / predicted2.shape[0]
print(accuracy2)

tensor(0.8575)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [ ]:
# Afisam acuratetea
predicted2 = torch.tensor(torch.argmax(model(valid_data2), dim=-1))
accuracy2 = torch.sum(predicted2 == valid_label2.ravel())
accuracy2 = accuracy2 / predicted2.shape[0]
print(accuracy2)

tensor(0.7554)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [ ]:
# Afisam matricea de confuzie
classification_matrix = confusion_matrix(predicted, valid_label2)
print(classification_matrix)

[[  4   4   3   1   0   0   0   0   0]
 [ 68 281 228  54   9   2   2   1   0]
 [ 40 135 119  20   4   0   1   0   1]
 [  3  38  19   4   0   0   0   0   0]
 [  0   3   4   0   0   0   0   0   0]
 [  2   2   4   1   0   0   0   0   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   1   0   0   0   0   0   0]]


# Regresie pe Pret




In [ ]:
# impartim datele si etichetele
X = data.drop(columns='Pret').values
Y = data[['Pret']].values.ravel()
train_data1, valid_data1, train_label1, valid_label1 = train_test_split(X, Y, train_size=0.7)


In [ ]:
# Normalizam datele 

std_scale = preprocessing.StandardScaler().fit(train_data1)
train_data1 = std_scale.transform(train_data1)
train_data1 = torch.tensor(train_data1).float()

valid_data1 = std_scale.transform(valid_data1)
valid_data1 = torch.tensor(valid_data1).float()

train_label1 = torch.tensor(train_label1).float()
valid_label1 = torch.tensor(valid_label1).float()

In [ ]:
#Alegem un learning rate si numarul de pasi de antrenare

lr = 0.1
total_steps = 500
model = GDLinearRegression()
optimizer = GD(model.parameters(), lr=lr)
criterion = MSE()
for i in range(total_steps):
    train(model, train_data1, train_label1, optimizer, criterion)

In [ ]:
# Afisam acuratetea de datele de train

with torch.no_grad():
    pred_label1 = model(train_data1)


predicted1 = torch.round(pred_label1).numpy()
train_label1 = train_label1.numpy()
accuracy1 = np.sum((predicted1 == train_label1))
accuracy1 = accuracy1 / predicted1.shape[0]
print("Acuratete pe date de train")
print(accuracy1)

Acuratete pe date de train
0.0


In [ ]:
# Afisam acuratetea de datele de validare

with torch.no_grad():
    pred_label1 = model(train_data1)


predicted1 = torch.round(pred_label1).numpy()
train_label1 = train_label1.numpy()
accuracy1 = np.sum((predicted1 == train_label1))
accuracy1 = accuracy1 / predicted1.shape[0]
print("Acuratete pe date de validare")
print(accuracy1)

AttributeError: ignored

# Matricile de confuzie

In [ ]:
print("Matricea pentru clasificare")
print(classification_matrix)
print("Matricea pentru regresie")
print(regression_matrix1)

Matricea pentru clasificare
[[  4   4   3   1   0   0   0   0   0]
 [ 68 281 228  54   9   2   2   1   0]
 [ 40 135 119  20   4   0   1   0   1]
 [  3  38  19   4   0   0   0   0   0]
 [  0   3   4   0   0   0   0   0   0]
 [  2   2   4   1   0   0   0   0   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   1   0   0   0   0   0   0]]
Matricea pentru regresie
[[ 12   0   0   0   0   0   0   0   0]
 [105 441  97   2   0   0   0   0   0]
 [  0  30 246  40   4   0   0   0   0]
 [  0   0  29  28   7   0   0   0   0]
 [  0   0   0   5   2   0   0   0   0]
 [  0   0   0   5   2   1   0   1   0]
 [  0   0   0   0   0   0   1   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0]]
